<a href="https://colab.research.google.com/github/SavageGinny/MLP-Jupiters/blob/main/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Загружаем библиотеки

In [ ]:
import numpy as np
import pandas as pd
import math

Текст

In [ ]:
df = pd.read_csv('dataset.csv')

In [ ]:
df

,x,y
0,-5,-9.0
1,-4,-6.5
2,-3,-5.0
3,-2,-2.8
4,-1,-1.2
5,0,1.0
6,1,2.8
7,2,5.1
8,3,6.9
9,4,9.2


Линейная регрессия

In [ ]:
def linear_regression(df: pd.DataFrame) -> list[float]:
    x_vals = df['x'].tolist()
    y_vals = df['y'].tolist()

    n = len(x_vals)
    mean_x = sum(x_vals) / n
    mean_y = sum(y_vals) / n

    # коэффициенты для линейной регрессии
    numerator = sum((x - mean_x) * (y - mean_y) for x, y in zip(x_vals, y_vals))
    denominator = sum((x - mean_x) ** 2 for x in x_vals)

    w = numerator / denominator
    b = mean_y - w * mean_x

    # прогнозы: y_pred = w * x + b
    y_pred = [w * x + b for x in x_vals]
    return y_pred

Функция активации



In [ ]:
def activation_func(x: list[float]) -> list[float]:
    def sigmoid(val):
        return 1 / (1 + math.exp(-val))

    return [sigmoid(i) for i in x]


Нейрон

In [ ]:
def neuron(df: pd.DataFrame):
    lin_output = linear_regression(df)
    activated_output = activation_func(lin_output)
    return activated_output

In [ ]:
neuron(df)

[0.0001376155381942285,
 0.0010049438286321476,
 0.007298745576231861,
 0.050997425343559194,
 0.2820034509256191,
 0.7416467954863152,
 0.9545069936941605,
 0.9935212462181178,
 0.9991085934824365,
 0.9998779440673917,
 0.999983298578152]

Класс

In [ ]:
class NeuralNetwork:
    def __init__(self, input_size, hidden1_size, hidden2_size, output_size, learning_rate=0.01):
        np.random.seed(42)
        self.lr = learning_rate

        self.W1 = np.random.randn(input_size, hidden1_size) * 0.1
        self.b1 = np.zeros((1, hidden1_size))

        self.W2 = np.random.randn(hidden1_size, hidden2_size) * 0.1
        self.b2 = np.zeros((1, hidden2_size))

        self.W3 = np.random.randn(hidden2_size, output_size) * 0.1
        self.b3 = np.zeros((1, output_size))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward(self, X):
        self.z1 = X @ self.W1 + self.b1
        self.a1 = self.sigmoid(self.z1)

        self.z2 = self.a1 @ self.W2 + self.b2
        self.a2 = self.sigmoid(self.z2)

        self.z3 = self.a2 @ self.W3 + self.b3
        self.output = self.sigmoid(self.z3)

        return self.output

    def backward(self, X, y):
        m = X.shape[0]
        output_error = self.output - y
        d_output = output_error * self.sigmoid_derivative(self.output)

        dW3 = self.a2.T @ d_output
        db3 = np.sum(d_output, axis=0, keepdims=True)

        d_a2 = d_output @ self.W3.T
        d_z2 = d_a2 * self.sigmoid_derivative(self.a2)
        dW2 = self.a1.T @ d_z2
        db2 = np.sum(d_z2, axis=0, keepdims=True)

        d_a1 = d_z2 @ self.W2.T
        d_z1 = d_a1 * self.sigmoid_derivative(self.a1)
        dW1 = X.T @ d_z1
        db1 = np.sum(d_z1, axis=0, keepdims=True)

        # Градиентный спуск
        self.W3 -= self.lr * dW3 / m
        self.b3 -= self.lr * db3 / m
        self.W2 -= self.lr * dW2 / m
        self.b2 -= self.lr * db2 / m
        self.W1 -= self.lr * dW1 / m
        self.b1 -= self.lr * db1 / m

    def train(self, df: pd.DataFrame, epochs=1000):
        X = df['x'].values.reshape(-1, 1)
        y = df['y'].values.reshape(-1, 1)

        for epoch in range(epochs):
            self.forward(X)
            self.backward(X, y)

            if epoch % 100 == 0:
                loss = np.mean((self.output - y) ** 2)
                print(f"Epoch {epoch}: Loss = {loss:.4f}")

    def predict(self, x_list: list[float]) -> list[float]:
        X = np.array(x_list).reshape(-1, 1)
        predictions = self.forward(X)
        return predictions.flatten().tolist()


In [ ]:
nn = NeuralNetwork(
    input_size=1,
    hidden1_size=4,
    hidden2_size=3,
    output_size=1,
    learning_rate=0.1
)

nn.train(df, epochs=1000)

predicted = nn.predict(df["x"].tolist())
print(predicted)

Epoch 0: Loss = 39.9296
Epoch 100: Loss = 39.6657
Epoch 200: Loss = 39.5832
Epoch 300: Loss = 39.3609
Epoch 400: Loss = 37.2259
Epoch 500: Loss = 35.3435
Epoch 600: Loss = 35.0134
Epoch 700: Loss = 34.9001
Epoch 800: Loss = 34.8443
Epoch 900: Loss = 34.8114
[0.013584085357690238, 0.013664155512815718, 0.014022984302632465, 0.015901761618484837, 0.03489495736590591, 0.7901805048095889, 0.9922176706526766, 0.9953559738112728, 0.9957527387728253, 0.9958334891817883, 0.995852003490326]
